# Election Page

> The page of the app that generates the data table view.

In [ ]:
#| default_exp pages/election_page

In [ ]:
#| export
# TODO minimise these imports a bit where possible
from nbdev.showdoc import *
# TODO work out how to get around below hack
try:
    from SolomonIslandsDataMap.dash_components import election_bar_plot, gen_dd
    from SolomonIslandsDataMap.app_data import mytitle, election_map, election_bar, stored_data, fake_slider
    from SolomonIslandsDataMap.load_data import SolomonGeo
except: 
    from dash_components import election_bar_plot, gen_dd
    from app_data import mytitle, election_map, election_bar, stored_data, fake_slider
    from load_data import SolomonGeo
from fastcore.test import *
from dash import Dash, dcc, callback, Output, Input, State, html, Patch, ctx, register_page, callback_context 
import dash_bootstrap_components as dbc    
import dash_ag_grid as dag
import plotly.graph_objects as go
from dash.exceptions import PreventUpdate

In [ ]:
#| export
try:
    register_page(__name__, 
                        path='/electionmap',
                        title='Election Map',
                        name='Election Map')
except:
    pass

Setup some of  the key data contiaining objects

In [ ]:
#| export
init_init = dcc.Store(id="initial-initial", data='election')
def layout():
    return dbc.Col(mytitle, width = 8),\
                dcc.Loading(
                    id="loading-election-map",
                    type="default",
                    children = election_map,
                ),\
                dcc.Loading(
                    id="loading-election-bar",
                    type="default",
                    children = election_bar,), init_init, fake_slider


## Global Variables

In [ ]:
#| export
dict_sol = stored_data.data

# Dash Callbacks
Create interactivity in the data table

In [ ]:
#| export
# Callback allows components to interact
@callback(
    Output('election_bar', 'figure'),
    Input("segmented_type", 'value'),
    Input('electionDropdown', 'value'),
    Input('locDropdown', 'value'),
    Input('initial-initial', 'data'),
    Input('elecYearDropdown', 'value'),
    State("segmented_geo", 'value'),
    #State('geo_df', 'data'),
    allow_duplicate=True,
    prevent_initial_call=True
)
def update_election_bar(data_type:str, # User input of type of data
                     election:str, # A string contiaining the election type
                     loc_selection:str, # The selected locations, may be none
                     init_load:{}, # An empty dictionary always, triggers initial load
                     elecYear:str, # The year of the election
                     geo_input:str, # User input from the geography dropdown
                     #dict_sol:dict, # The dataset in dictionary form
              )->(type(go.Figure())): # Returns a graph object figure after being updated and the dynamic title
    '''
    Updates the focus census variable or geography dispalayed on the map
    '''
    sol_geo = SolomonGeo.gen_stored(dict_sol) # reload the data

    # Create newly selected barplot
    print("Func: update_election_bargraph")
    loc = 'Central Honiara'
    if len(loc_selection) > 0: 
        loc = loc_selection[-1]

    bg = election_bar_plot(sol_geo, geo_input, elecYear, election, loc, data_type)

    # returned objects are assigned to the component property of the Output
    # After updating fileter, we always reset map selection 

    return  bg

### Election Map Click
Update the bar graph base on the map click

In [ ]:
#| export
@callback(
    Output('locDropdown', 'value', allow_duplicate=True),
    Output('election_map', "clickData"),
    Output('election_map', "selectedData"),
    Input('election_map', 'clickData'),
    State('locDropdown', 'value'),
    prevent_initial_call=True,
    allow_duplicate=True,
) # TODO - eventually can I turn this into a clientside callbback?
def map_click(clickData:dict, # The currently clicked location on the map
              selectedData:dict, # The currently selected locations on the map
                )->[str]: # Returns the new value for the dropdown
    """This function updates the elections barplot based on the map click data,
    only a single location selected for this one"""

    print("Func: map_click")
    if clickData is None:
        # When nothing is selected or clicked, don't update anything
        raise PreventUpdate # if changing to clientside, use: window.dash_clientside.no_update
        #return prev_locs, None, None
    else:
        selections = list(map(lambda x: x['location'], clickData['points']))
        # Return the last selection
        return selections, None, None

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()